In [34]:
import math
import random
import sys
fib = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049]

In [35]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def getDistance(self, p):
        return math.floor(math.sqrt((self.x - p.x)**2 + (self.y - p.y)**2))

    def __str__(self) -> str:
        return 'x: ' + str(self.x) + ' '*max(0, len(str(self.x)) - 3) + '\ty: ' + str(self.y) + ' '*max(0, len(str(self.x)) - 5) + '\t'


class Perso(Point):
    def __init__(self, x, y):
        Point.__init__(self, x, y)
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return super().__str__()


class Human(Point):
    def __init__(self, id, x, y):
        Point.__init__(self, x, y)
        self.x = x
        self.y = y
        self.id = id

    def __str__(self) -> str:
        return super().__str__() + 'human id: ' + str(self.id)


class Zombie(Point):
    def __init__(self, id, x, y):
        Point.__init__(self, x, y)
        self.id = id
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return super().__str__() + 'zombie id: ' + str(self.id)

In [177]:
class Tour:
    def __init__(self, i, ash, humans_list, zombies_list, score, out_x, out_y):
        self.i = i
        self.ash = ash
        self.humans_list  = humans_list
        self.zombies_list = zombies_list
        self.score = score
        self.out_x = out_x
        self.out_y = out_y
    
    def getClosestZombie(self, point, z_list):
        if z_list != []:
            return sorted(z_list, key=lambda x: x.getDistance(point))[0]
    
    def getClosestHuman(self, point):
        return sorted([h for h in self.humans_list], key=lambda x: x.getDistance(point))[0]
    
    def next_zombie_position(self, z):
        human_cible = self.getClosestHuman(z)
        if human_cible.getDistance(z) >= 400:
            angle = math.atan2(human_cible.y - z.y, human_cible.x - z.x)
            nx = math.floor(z.x + 400*math.cos(angle))
            ny = math.floor(z.y + 400*math.sin(angle))
        else:
            nx,ny = human_cible.x, human_cible.y
        return [nx, ny]
    
    def getNextZombiesPositions(self):
        r=[]
        for z in self.zombies_list:
            [x,y] = self.next_zombie_position(z)
            r.append(Zombie(z.id, x, y))
        return r
    
    def getNextAsh(self, ash_mx, ash_my):
        if self.ash.getDistance(Point(ash_mx, ash_my)) >= 1000:
            angle = math.atan2(ash_my - self.ash.y, ash_mx - self.ash.x)
            nx = math.floor(self.ash.x + 1000*math.cos(angle))
            ny = math.floor(self.ash.y + 1000*math.sin(angle))
        else:
            nx, ny = ash_mx, ash_my
        return Perso(nx, ny)
    
    def getNextZombiesList(self, z_list, ash): # ajout un argument ash (self.getNextAsh(ash_mx, ash_my) lorsqu'on l'appellera)
        index = 0
        r = []
        for z in z_list:
            if ash.getDistance(z) >= 2000:
                r.append(Zombie(z.id, z.x, z.y))
            else:
                z_value = 10*len(self.humans_list[1:])**2*fib[index+2]
                index += 1
                self.score += z_value
        return r

    def getNextHumanList(self, z_list):
        r = []
        for h in self.humans_list:
            if self.getClosestZombie(h,z_list).getDistance(h) != 0:
                if h.id==0:
                    h.x = self.getNextAsh(self.out_x, self.out_y).x
                    h.y = self.getNextAsh(self.out_x, self.out_y).y
                r.append(h)
        return r
    
    def getNextTurn(self, out_x, out_y):
        nextAsh = self.getNextAsh(4000,1000)
        z_moved = self.getNextZombiesPositions()
        z_survivals = self.getNextZombiesList(z_moved, self.getNextAsh(out_x, out_y))
        if z_survivals == []:
            # 0 zombie, tous les humains du tour précédant survivent
            h_survivals = self.humans_list
        else:
            h_survivals = self.getNextHumanList(z_survivals)

        if z_survivals != [] and h_survivals[1:] != []:
            return Tour(self.i + 1, nextAsh, h_survivals, z_survivals, self.score, 4000, 1000)
        else:
            if h_survivals[1:] == []:
                print('lose')
            else:
                print('win')

    def allo(self):
        print('---------- tour ' + str(self.i)  + ' -----('+ str(self.score)  + ')--------------------------', file=sys.stderr, flush=True)
        # print('x,y = ' + str(x) + ',' + str(y), file=sys.stderr, flush=True)
        print('[human_count, zombie_count] = [' + str(len(self.humans_list)-1) + ',' + str(len(self.zombies_list)) + ']', file=sys.stderr, flush=True)
        s = 'humans_list  = ['
        for h in self.humans_list:
            s += 'Human(' + str(h.id) + ',' + str(h.x) + ',' + str(h.y) + '), '
        s = s[:-2] + ']'
        print(s, file=sys.stderr, flush=True)
        print('---', file=sys.stderr, flush=True)
        s = 'zombies_list  = ['
        for z in self.zombies_list:
            s += 'Zombie(' + str(z.id) + ',' + str(z.x) + ',' + str(z.y) + '), '
        s = s[:-2] + ']'
        print(s, file=sys.stderr, flush=True)
        print('----------------------------------------------------------', file=sys.stderr, flush=True)



In [178]:
x,y = 4000,1000
[human_count, zombie_count] = [2,16]
humans_list  = [Human(0,4000,1000), Human(1,0,1000), Human(2,0,8000)]
zombies_list  = [Zombie(0,5000,1000), Zombie(1,5000,8000), Zombie(2,7000,1000), Zombie(3,7000,8000), Zombie(4,9000,1000), Zombie(5,9000,8000), Zombie(6,11000,1000), Zombie(7,11000,8000), Zombie(8,13000,1000), Zombie(9,13000,8000), Zombie(10,14000,1000), Zombie(11,14000,8000), Zombie(12,14500,1000), Zombie(13,14500,8000), Zombie(14,15000,1000), Zombie(15,15000,8000)]
t0 = Tour(0, Perso(x,y), humans_list, zombies_list, 0,  4000, 1000)
t0.allo()
t0.getNextTurn(4000,1000).allo()


---------- tour 0 -----(0)--------------------------
[human_count, zombie_count] = [2,16]
humans_list  = [Human(0,4000,1000), Human(1,0,1000), Human(2,0,8000)]
---
zombies_list  = [Zombie(0,5000,1000), Zombie(1,5000,8000), Zombie(2,7000,1000), Zombie(3,7000,8000), Zombie(4,9000,1000), Zombie(5,9000,8000), Zombie(6,11000,1000), Zombie(7,11000,8000), Zombie(8,13000,1000), Zombie(9,13000,8000), Zombie(10,14000,1000), Zombie(11,14000,8000), Zombie(12,14500,1000), Zombie(13,14500,8000), Zombie(14,15000,1000), Zombie(15,15000,8000)]
----------------------------------------------------------
---------- tour 1 -----(40)--------------------------
[human_count, zombie_count] = [2,15]
humans_list  = [Human(0,4000,1000), Human(1,0,1000), Human(2,0,8000)]
---
zombies_list  = [Zombie(1,4600,8000), Zombie(2,6600,1000), Zombie(3,6600,8000), Zombie(4,8600,1000), Zombie(5,8767,7674), Zombie(6,10600,1000), Zombie(7,10717,7717), Zombie(8,12600,1000), Zombie(9,12684,7754), Zombie(10,13600,1000), Zombie(11,

In [179]:
x,y = 15000,8000
[human_count, zombie_count] = [1,1]
humans_list  = [Human(0,4000,1000), Human(2,0,8000)]
zombies_list  = [Zombie(0,450,8000)]
t0 = Tour(0, Perso(x,y), humans_list, zombies_list, 0,  4000, 1000)
t0.allo()
t0.getNextTurn(4000,1000).allo()
t0.getNextTurn(4000,1000).getNextTurn(4000,1000)

---------- tour 0 -----(0)--------------------------
[human_count, zombie_count] = [1,1]
humans_list  = [Human(0,4000,1000), Human(2,0,8000)]
---
zombies_list  = [Zombie(0,450,8000)]
----------------------------------------------------------
---------- tour 1 -----(0)--------------------------
[human_count, zombie_count] = [1,1]
humans_list  = [Human(0,14156,7463), Human(2,0,8000)]
---
zombies_list  = [Zombie(0,50,8000)]
----------------------------------------------------------


lose
